# Open Contextual RAG

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Open_Contextual_RAG.ipynb)

## Introduction

[Contextual Retrieval](https://www.anthropic.com/news/contextual-retrieval) is a chunk augmentation technique that uses a LLM to ehance each chunk.

<img src="images/cRAG.png" width="1000">

Here's an overview of how it works. 👇

Contextual RAG:

1. For every chunk - prepend an explanatory context snippet that situates the chunk within the rest
 of the document. -> Get a small cost effective LLM to do this.

2. Hybrid Search: Embed the chunk using both sparse (keyword) and dense(semantic) embeddings.

3. Perform rank fusion using an algorithm like Reciprocal Rank Fusion(RRF).

4. Retrieve top 150 chunks and pass those to a Reranker to obtain top 20 chunks.

5. Pass top 20 chunks to LLM to generate an answer.

Below we implement each step in this process using Open Source models.

To breakdown the concept further we break down the process into a one-time indexing step and a query time step.

Data Ingestion Phase:

<img src="images/cRAG_indexing.png" width="500">

1. Data processing and chunking
2. Context generation using a quantized Llama 3B Model
3. Vector Embedding and Index Generation
4. BM25 Keyword Index Generation


At Query Time:

<img src="images/cRAG_querytime.png" width="750">

1. Perform retreival using both indices and combine them using RRF
2. Reranker to improve retreival quality
3. Generation with Llama 405b

### Install Libraries

In [2]:
!pip install together # To access open source LLMs
!pip install --upgrade tiktoken # To count total token counts
!pip install beautifulsoup4 # To scrape documents to RAG over
!pip install bm25s # To implement out key-word BM25 search

### 1. Data Processing and Chunking

We will RAG over Paul Grahams latest essay titled [**Founder Mode**](https://paulgraham.com/foundermode.html).

In [3]:
# Let's download the essay from Paul Graham's website

import requests
from bs4 import BeautifulSoup

def scrape_pg_essay():

    url = 'https://paulgraham.com/foundermode.html'

    try:
        # Send GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Paul Graham's essays typically have the main content in a font tag
        # You might need to adjust this selector based on the actual HTML structure
        content = soup.find('font')

        if content:
            # Extract and clean the text
            text = content.get_text()
            # Remove extra whitespace and normalize line breaks
            text = ' '.join(text.split())
            return text
        else:
            return "Could not find the main content of the essay."

    except requests.RequestException as e:
        return f"Error fetching the webpage: {e}"

# Scrape the essay
pg_essay = scrape_pg_essay()

In [6]:
print(pg_essay[:1000])

September 2024At a YC event last week Brian Chesky gave a talk that everyone who was there will remember. Most founders I talked to afterward said it was the best they'd ever heard. Ron Conway, for the first time in his life, forgot to take notes. I'm not going to try to reproduce it here. Instead I want to talk about a question it raised.The theme of Brian's talk was that the conventional wisdom about how to run larger companies is mistaken. As Airbnb grew, well-meaning people advised him that he had to run the company in a certain way for it to scale. Their advice could be optimistically summarized as "hire good people and give them room to do their jobs." He followed this advice and the results were disastrous. So he had to figure out a better way on his own, which he did partly by studying how Steve Jobs ran Apple. So far it seems to be working. Airbnb's free cash flow margin is now among the best in Silicon Valley.The audience at this event included a lot of the most successful fo

In [7]:
# We can get away with naive fixed sized chunking as the context generation will add meaning to these chunks

def create_chunks(document, chunk_size=300, overlap=50):
    return [document[i : i + chunk_size] for i in range(0, len(document), chunk_size - overlap)]

In [8]:
chunks = create_chunks(pg_essay, chunk_size=250, overlap=30)

for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}: {chunk}")

Chunk 1: September 2024At a YC event last week Brian Chesky gave a talk that everyone who was there will remember. Most founders I talked to afterward said it was the best they'd ever heard. Ron Conway, for the first time in his life, forgot to take notes. I'
Chunk 2: life, forgot to take notes. I'm not going to try to reproduce it here. Instead I want to talk about a question it raised.The theme of Brian's talk was that the conventional wisdom about how to run larger companies is mistaken. As Airbnb grew, well-me
Chunk 3: taken. As Airbnb grew, well-meaning people advised him that he had to run the company in a certain way for it to scale. Their advice could be optimistically summarized as "hire good people and give them room to do their jobs." He followed this advice
Chunk 4: jobs." He followed this advice and the results were disastrous. So he had to figure out a better way on his own, which he did partly by studying how Steve Jobs ran Apple. So far it seems to be working. Airbnb's 

### 2. Generating Contextual Chunks

This part contains the main intuition behind `Contextual Retrieval`. We will make an LLM call for each chunk to add much needed relevant context to the chunk. In order to do this we pass in the ENTIRE document per LLM call.

It may seem that passing in the entire document per chunk and making an LLM call per chunk is quite inefficient, this is true and there very well might be more efficient techniques to accomplish the same end goal. But in keeping with implementing the current technique at hand lets do it.

Additionally using quantized small 1-3b models (here we will use Llama 3b) along with prompt caching does make this more feasible.

Prompt caching allows key and value matrices corresponding to the document to be cached for future LLM calls.

In [10]:
import together, os
from together import Together

# Paste in your Together AI API Key or load it
TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")
client = Together(api_key = TOGETHER_API_KEY)

In [11]:
# We want to generate a snippet explaining the relevance/importance of the chunk with
# full document in mind.

CONTEXTUAL_RAG_PROMPT = """
Given the document below, we want to explain what the chunk captures in the document.

{WHOLE_DOCUMENT}

Here is the chunk we want to explain:

{CHUNK_CONTENT}

Answer ONLY with a succinct explaination of the meaning of the chunk in the context of the whole document above.
"""

In [12]:
from typing import List

# First we will just generate the prompts and examine them

def generate_prompts(document : str, chunks : List[str]) -> List[str]:
  prompts = []
  for chunk in chunks:
    prompt = CONTEXTUAL_RAG_PROMPT.format(WHOLE_DOCUMENT=document, CHUNK_CONTENT=chunk)
    prompts.append(prompt)
  return prompts

In [13]:
prompts = generate_prompts(pg_essay, chunks)

In [15]:
print(prompts[0][6500:])

 even things they should will use founder mode as the excuse. Or managers who aren't founders will decide they should try to act like founders. That may even work, to some extent, but the results will be messy when it doesn't; the modular approach does at least limit the damage a bad CEO can do.Thanks to Brian Chesky, Patrick Collison, Ron Conway, Jessica Livingston, Elon Musk, Ryan Petersen, Harj Taggar, and Garry Tan for reading drafts of this.

Here is the chunk we want to explain:

September 2024At a YC event last week Brian Chesky gave a talk that everyone who was there will remember. Most founders I talked to afterward said it was the best they'd ever heard. Ron Conway, for the first time in his life, forgot to take notes. I'

Answer ONLY with a succinct explaination of the meaning of the chunk in the context of the whole document above.



#### How much does generating one Contextual chunk cost?

Below we'll look at total tokens per prompt and estimate the cost of generating one/many contextual chunks.

In [16]:
# token counter function
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [17]:
# We'll assume we're using the "cl100k_base" encoding

num_tokens_from_string(pg_essay, "cl100k_base")

1457

In [18]:
num_tokens_from_string(prompts[0], "cl100k_base")

1565

In [96]:
#import python mean function
from statistics import mean

mean([num_tokens_from_string(prompt, "cl100k_base") for prompt in prompts])

1559.21875

In [120]:
print(f"Cost to generate contextual chunks = ${32*0.06*(1660/1000000)}")
print(f"Number of Chunks Processed with $1.00 = {1000000/(0.06*1660)} chunks")

Cost to generate contextual chunks = $0.0031872
Number of Chunks Processed with $1.00 = 10040.160642570281 chunks


`$0.06` per 1 million tokens for Llama 3b.

Assuming input lenght of ~ 1560 tokens and output length of 100 tokens.

Given an approximate token count of ~ 1660 per context generation we can generate 10,000 contexts for a $1.00.

Even if we were using the full context length of the model(~130,000) for each context generation we could generate ~128 contexts for $1.00.

In [20]:
def generate_context(prompt: str):
    """
    Generates a contextual response based on the given prompt using the specified language model.
    Args:
        prompt (str): The input prompt to generate a response for.
    Returns:
        str: The generated response content from the language model.
    """
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=1
    )
    return response.choices[0].message.content

In [21]:
# Lets generate one context and concatenate it with its chunk to see what the output looks like
context_0 = generate_context(prompts[0])
print(context_0 + " \n\n" + chunks[0])

This chunk introduces the topic of the article, discussing a recent talk by Brian Chesky at a Y Combinator (YC) event and the impressions and reactions of founders attending the event, setting the stage for exploring differing wisdom on how to run larger companies. 

September 2024At a YC event last week Brian Chesky gave a talk that everyone who was there will remember. Most founders I talked to afterward said it was the best they'd ever heard. Ron Conway, for the first time in his life, forgot to take notes. I'


In [24]:
# Let's generate the entire list of contextual chunks

contextual_chunks = [generate_context(prompts[i])+' '+chunks[i] for i in range(len(chunks))]

In [28]:
# We should have one contextual chunk for each original chunk
len(contextual_chunks), len(contextual_chunks) == len(chunks)

(32, True)

### 3. Embedding Generation

We will now use `bge-large-en-v1.5` to embed the augmented chunks above into a vector index.

In [29]:
from typing import List
import together
import numpy as np

def generate_embeddings(input_texts: List[str], model_api_string: str) -> List[List[float]]:
    """Generate embeddings from Together python library.

    Args:
        input_texts: a list of string input texts.
        model_api_string: str. An API string for a specific embedding model of your choice.

    Returns:
        embeddings_list: a list of embeddings. Each element corresponds to the each input text.
    """
    outputs = client.embeddings.create(
        input=input_texts,
        model=model_api_string,
    )
    return [x.embedding for x in outputs.data]

In [30]:
contextual_embeddings = generate_embeddings(contextual_chunks, "BAAI/bge-large-en-v1.5")

In [31]:
# Each vector is 1024 dimensional

len(contextual_embeddings[0])

1024

In [34]:
# Generate the vector embeddings for the query
query = "What are 'skip-level' meetings?"

query_embedding = generate_embeddings([query], 'BAAI/bge-large-en-v1.5')[0]

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between the query embedding and each movie embedding
similarity_scores = cosine_similarity([query_embedding], contextual_embeddings)
indices = np.argsort(-similarity_scores)

In [50]:
top_5_indices = indices[0][:5]
top_5_indices

array([17, 18, 11, 26, 27])

In [38]:
top_5_chunks = [contextual_chunks[index] for index in indices[0]][:5]

top_5_chunks

['This chunk discusses the shift in company management away from the "manager mode" that most companies follow, where CEOs engage with the company only through their direct reports, to "founder mode", where CEOs engage more directly with even higher-level employees and potentially skip over direct reports, potentially leading to "skip-level" meetings. ts of, it\'s pretty clear that it\'s going to break the principle that the CEO should engage with the company only via his or her direct reports. "Skip-level" meetings will become the norm instead of a practice so unusual that there\'s a name for it. An',
 'This chunk refers to "skip-level" meetings, which are a key characteristic of founder mode, where the CEO engages directly with the company beyond their direct reports. This contrasts with the "manager mode" of addressing company issues, where decisions are made perfunctorily via a hierarchical system, to which founders instinctively rebel. that there\'s a name for it. And once you aba

In [70]:
def vector_retreival(query: str, top_k: int = 5, vector_index: np.ndarray = None) -> List[int]:
    """
    Retrieve the top-k most similar items from an index based on a query.
    Args:
        query (str): The query string to search for.
        top_k (int, optional): The number of top similar items to retrieve. Defaults to 5.
        index (np.ndarray, optional): The index array containing embeddings to search against. Defaults to None.
    Returns:
        List[int]: A list of indices corresponding to the top-k most similar items in the index.
    """

    query_embedding = generate_embeddings([query], 'BAAI/bge-large-en-v1.5')[0]
    similarity_scores = cosine_similarity([query_embedding], vector_index)

    return list(np.argsort(-similarity_scores)[0][:top_k])

In [71]:
vector_retreival(query = "What are 'skip-level' meetings?", top_k = 5, vector_index = contextual_embeddings)

[17, 18, 11, 26, 27]

We now have a way to retrieve from the vector index given a query.

### 4. BM25 Search Using `bm25s` library

Lets build a keyword index that allows us to use BM25 to perform lexical search based on the words present in the query and the contextual chunks.

In [43]:
import bm25s

# Create the BM25 model and index the corpus
retriever = bm25s.BM25(corpus=contextual_chunks)
retriever.index(bm25s.tokenize(contextual_chunks))

Split strings:   0%|          | 0/32 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/32 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/32 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
# Query the corpus and get top-k results
query = "What are 'skip-level' meetings?"
results, scores = retriever.retrieve(bm25s.tokenize(query), k=5,)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
for doc in results[0]:
  print(f"Chunk Index {contextual_chunks.index(doc)} : {doc}")

Chunk Index 17 : This chunk discusses the shift in company management away from the "manager mode" that most companies follow, where CEOs engage with the company only through their direct reports, to "founder mode", where CEOs engage more directly with even higher-level employees and potentially skip over direct reports, potentially leading to "skip-level" meetings. ts of, it's pretty clear that it's going to break the principle that the CEO should engage with the company only via his or her direct reports. "Skip-level" meetings will become the norm instead of a practice so unusual that there's a name for it. An
Chunk Index 18 : This chunk refers to "skip-level" meetings, which are a key characteristic of founder mode, where the CEO engages directly with the company beyond their direct reports. This contrasts with the "manager mode" of addressing company issues, where decisions are made perfunctorily via a hierarchical system, to which founders instinctively rebel. that there's a name 

In [60]:
def bm25_retreival(query: str, k : int, bm25_index) -> List[int]:
    """
    Retrieve the top-k document indices based on the BM25 algorithm for a given query.
    Args:
        query (str): The search query string.
        k (int): The number of top documents to retrieve.
        bm25_index: The BM25 index object used for retrieval.
    Returns:
        List[int]: A list of indices of the top-k documents that match the query.
    """

    results, scores = bm25_index.retrieve(bm25s.tokenize(query), k=k)

    return [contextual_chunks.index(doc) for doc in results[0]]

In [61]:
bm25_retreival(query = "What are 'skip-level' meetings?", k = 5, bm25_index = retriever)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

[17, 18, 11, 22, 26]

## Everything below this point will happen at query time!

<img src="images/cRAG_querytime.png" width="750">

### 1. Perform Vector and BM25 retrieval and then fuse the ranks using the RRF algorithm implemented below.

In [73]:
from collections import defaultdict

def reciprocal_rank_fusion(*list_of_list_ranks_system, K=60):
    """
    Fuse rank from multiple IR systems using Reciprocal Rank Fusion.

    Args:
    * list_of_list_ranks_system: Ranked results from different IR system.
    K (int): A constant used in the RRF formula (default is 60).

    Returns:
    Tuple of list of sorted documents by score and sorted documents
    """
    # Dictionary to store RRF mapping
    rrf_map = defaultdict(float)

    # Calculate RRF score for each result in each list
    for rank_list in list_of_list_ranks_system:
        for rank, item in enumerate(rank_list, 1):
            rrf_map[item] += 1 / (rank + K)

    # Sort items based on their RRF scores in descending order
    sorted_items = sorted(rrf_map.items(), key=lambda x: x[1], reverse=True)

    # Return tuple of list of sorted documents by score and sorted documents
    return sorted_items, [item for item, score in sorted_items]

# Example ranked lists from different sources
vector_top_k = vector_retreival(query = "What are 'skip-level' meetings?", top_k = 5, vector_index = contextual_embeddings)
bm25_top_k = bm25_retreival(query = "What are 'skip-level' meetings?", k = 5, bm25_index = retriever)


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [79]:
# Combine the lists using RRF
hybrid_top_k = reciprocal_rank_fusion(vector_top_k, bm25_top_k)
hybrid_top_k[1]

[17, 18, 11, 26, 22, 27]

In [80]:
# Notice the reordering of the chunks based on the RRF score

for index in hybrid_top_k[1]:
  print(f"Chunk Index {index} : {contextual_chunks[index]}")

Chunk Index 17 : This chunk discusses the shift in company management away from the "manager mode" that most companies follow, where CEOs engage with the company only through their direct reports, to "founder mode", where CEOs engage more directly with even higher-level employees and potentially skip over direct reports, potentially leading to "skip-level" meetings. ts of, it's pretty clear that it's going to break the principle that the CEO should engage with the company only via his or her direct reports. "Skip-level" meetings will become the norm instead of a practice so unusual that there's a name for it. An
Chunk Index 18 : This chunk refers to "skip-level" meetings, which are a key characteristic of founder mode, where the CEO engages directly with the company beyond their direct reports. This contrasts with the "manager mode" of addressing company issues, where decisions are made perfunctorily via a hierarchical system, to which founders instinctively rebel. that there's a name 

In [81]:
hybrid_top_k_docs = [contextual_chunks[index] for index in hybrid_top_k[1]]

In [82]:
[contextual_chunks[index] for index in hybrid_top_k[1]]

['This chunk discusses the shift in company management away from the "manager mode" that most companies follow, where CEOs engage with the company only through their direct reports, to "founder mode", where CEOs engage more directly with even higher-level employees and potentially skip over direct reports, potentially leading to "skip-level" meetings. ts of, it\'s pretty clear that it\'s going to break the principle that the CEO should engage with the company only via his or her direct reports. "Skip-level" meetings will become the norm instead of a practice so unusual that there\'s a name for it. An',
 'This chunk refers to "skip-level" meetings, which are a key characteristic of founder mode, where the CEO engages directly with the company beyond their direct reports. This contrasts with the "manager mode" of addressing company issues, where decisions are made perfunctorily via a hierarchical system, to which founders instinctively rebel. that there\'s a name for it. And once you aba

### 2. Reranker To improve Quality

We will pass in the top 6 documents output from Hybrid search into a reranker to sort based on relevance to the query.

In [86]:
from together import Together

query = "What are 'skip-level' meetings?" # we keep the same query - can change if we want

response = client.rerank.create(
  model="Salesforce/Llama-Rank-V1",
  query=query,
  documents=hybrid_top_k_docs,
  top_n=3 # we only want the top 3 results
)

for result in response.results:
    print(f"Document: {hybrid_top_k_docs[result.index]}")
    print(f"Relevance Score: {result.relevance_score}\n\n")

Document: This chunk refers to "skip-level" meetings, which are a key characteristic of founder mode, where the CEO engages directly with the company beyond their direct reports. This contrasts with the "manager mode" of addressing company issues, where decisions are made perfunctorily via a hierarchical system, to which founders instinctively rebel. that there's a name for it. And once you abandon that constraint there are a huge number of permutations to choose from.For example, Steve Jobs used to run an annual retreat for what he considered the 100 most important people at Apple, and these wer
Relevance Score: 0.9733742140554578


Document: This chunk discusses the shift in company management away from the "manager mode" that most companies follow, where CEOs engage with the company only through their direct reports, to "founder mode", where CEOs engage more directly with even higher-level employees and potentially skip over direct reports, potentially leading to "skip-level" meetin

In [90]:
# Lets add the top 3 documents to a string

retreived_chunks = ''

for result in response.results:
    retreived_chunks += hybrid_top_k_docs[result.index] + '\n\n'

print(retreived_chunks)

This chunk refers to "skip-level" meetings, which are a key characteristic of founder mode, where the CEO engages directly with the company beyond their direct reports. This contrasts with the "manager mode" of addressing company issues, where decisions are made perfunctorily via a hierarchical system, to which founders instinctively rebel. that there's a name for it. And once you abandon that constraint there are a huge number of permutations to choose from.For example, Steve Jobs used to run an annual retreat for what he considered the 100 most important people at Apple, and these wer

This chunk discusses the shift in company management away from the "manager mode" that most companies follow, where CEOs engage with the company only through their direct reports, to "founder mode", where CEOs engage more directly with even higher-level employees and potentially skip over direct reports, potentially leading to "skip-level" meetings. ts of, it's pretty clear that it's going to break the

### 3. Call Generative Model - Llama 405b

We will pass the finalized 3 chunks into an LLM to get our final answer.

In [ ]:
# Generate a story based on the top 10 most similar movies

query = "What are 'skip-level' meetings?"

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    messages=[
      {"role": "system", "content": "You are a helpful chatbot."},
      {"role": "user", "content": f"Answer the question: {query}. Here is relevant information: {retreived_chunks}"},
    ],
)

In [93]:
response.choices[0].message.content

'"Skip-level" meetings refer to a management practice where a CEO or high-level executive engages directly with employees who are not their direct reports, bypassing the traditional hierarchical structure of the organization. This approach is characteristic of "founder mode," where the CEO seeks to have a more direct connection with the company beyond their immediate team. In contrast to the traditional "manager mode," where decisions are made through a hierarchical system, skip-level meetings allow for more open communication and collaboration between the CEO and various levels of employees. This approach is often used by founders who want to stay connected to the company\'s operations and culture, and to foster a more flat and collaborative organizational structure.'

Above we implemented Contextual Retrieval as discussed in [Anthropic's blog](https://www.anthropic.com/news/contextual-retrieval) using fully open source models! 

If you want to learn more about how to best use open models refer to our [docs](https://docs.together.ai/docs) here!